In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import os
import tensorflow.keras as keras

data: dict
epochs_ = 200
lr = 0.001
kernel = (2,2)
size_images = (192, 256)

In [11]:
import numpy as np

def index_genre(genre, genres):
    for (g,index) in zip(genres, range(len(genres))):
        if(g == genre):
            return index
    return -1

def get_data(data_path, genres, decoder, training_percentaje=0.6, validation_percentaje=0.2, test_percentaje=0.2):
    """ 
    data_path: se le pasa la direccion de la carpeta donde se encuentra la base de datos.
    genres: se le pasa una lista con los nombres da cada carpeta que contiene un genero dado.    
    decoder: funcion para decodificar el dato que se le pasa, por ejemplo en caso de imagenes habria hacer imread
    """
    
    data_training = {'in': [], 'out': []}
    data_validation = {'in': [], 'out': []}
    data_test = {'in': [], 'out': []}

    for genre in genres:
        files = os.listdir(data_path + genre)
        count = len(files)

        for (filename, index) in zip(files, range(len(files))):
            filepath = data_path + genre + '/' + filename

            if (index < training_percentaje * count):
                data_training['in'].append(decoder(filepath))
                data_training['out'].append(index_genre(genre, genres))
                continue

            if (index < (training_percentaje + validation_percentaje) * count):
                data_validation['in'].append(decoder(filepath))
                data_validation['out'].append(index_genre(genre, genres))
            else:
                data_test['in'].append(decoder(filepath))
                data_test['out'].append(index_genre(genre, genres))

    data_training = {'in': np.array(data_training['in']),'out': np.array(data_training['out'])}            
    data_validation = {'in': np.array(data_validation['in']),'out': np.array(data_validation['out'])}            
    data_test = {'in': np.array(data_test['in']),'out': np.array(data_test['out'])}            

    return {
        'data_training': data_training,
        'data_validation': data_validation,
        'data_testing': data_test
    }

In [18]:
import cv2

def read_image(filepath):
    global size_images
    image = cv2.imread(filepath)
    return cv2.resize(image, size_images)

def read_gray_image(filepath):
    global size_images
    img_gray = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

    # Convierte la matriz de la imagen a un tipo de datos float32
    # img_gris = img_gris.astype('float32')

    # Agrega una dimensión adicional parael canal de la imagen
    img_gray = np.expand_dims(img_gray, axis=-1)

    return cv2.resize(img_gray, size_images)

genres = ['blues', 'classical', 'country', 'disco',
          'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

data_path = '/content/drive/MyDrive/MFCC_dataset/'

def get_data_mfcc(training_percentaje=0.6, validation_percentaje=0.2, test_percentaje=0.2):
    return get_data(data_path, genres, read_image,training_percentaje,validation_percentaje,test_percentaje)

In [30]:
def data_internal():
    global data
    data = get_data_mfcc(training_percentaje=0.8, validation_percentaje=0.1, test_percentaje=0.1)

def training():
    global data
    global epochs_
    global size_images
    global lr

    data_internal()
    training_data = data['data_training']
    v_data = data['data_validation']

    model = mfcc_cnn_model(input_shape=(
        size_images[1], size_images[0], 3), classes=10, filter_shape=kernel)

    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(),
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        metrics=['accuracy'])

    checkpoint = keras.callbacks.ModelCheckpoint(
        '/content/drive/MyDrive/mfcc_models/best_weights.h5', 
        save_weights_only=True, 
        save_best_only=True, 
        monitor='val_accuracy', 
        mode='max')

    model.fit(
        training_data['in'], training_data['out'],
        validation_data=(v_data['in'], v_data['out']),
        epochs=epochs_,
        callbacks=[checkpoint]
        )

    model.save('/content/drive/MyDrive/mfcc_models/mfcc_model.h5')

def testing(model_name="mfcc_model.h5", epoch_name = 'best_weights.h5'):
    global data
    global size_images

    data_internal()
    input = data['data_testing']['in']
    output = data['data_testing']['out']

    path = '/content/drive/MyDrive/mfcc_models/' + model_name
    path_best_epoch = '/content/drive/MyDrive/mfcc_models/' + epoch_name

    model = keras.models.load_model(path)
    # model.load_weights(path_best_epoch)

    score = model.evaluate(input, output, verbose=0)
    # print('testing loss: ' + str(score[0]))
    print('testing accuracy: ' + str(score[1]))

    input = data['data_validation']['in']
    output = data['data_validation']['out']
    score = model.evaluate(input, output, verbose=0)
    # print('validation loss: ' + str(score[0]))
    print('validation accuracy: ' + str(score[1]))

    input = data['data_training']['in']
    output = data['data_training']['out']
    score = model.evaluate(input, output, verbose=0)
    # print('training loss: ' + str(score[0]))
    print('training accuracy: ' + str(score[1]))


In [14]:
def mfcc_cnn_model(input_shape=(128, 96, 1), classes=10, filter_shape=(3, 3)):
    X_input = keras.layers.Input(input_shape)

    X = keras.layers.Conv2D(32, filter_shape, activation='relu')(X_input)

    X = keras.layers.Conv2D(64, filter_shape, activation='relu')(X_input)
    X = keras.layers.AveragePooling2D((2, 2))(X)

    X = keras.layers.Conv2D(128, filter_shape, activation='relu')(X)
    X = keras.layers.AveragePooling2D((2, 2))(X)

    X = keras.layers.Conv2D(256, filter_shape, activation='relu')(X)
    X = keras.layers.GlobalAveragePooling2D()(X)

    X = keras.layers.Dense(256, activation='relu')(X)
    X = keras.layers.Dense(128, activation='relu')(X)
    X = keras.layers.Dense(64, activation='relu')(X)
    X = keras.layers.Dense(32, activation='relu')(X)
    X = keras.layers.Dense(classes, activation='softmax')(X)

    return keras.models.Model(inputs=X_input, outputs=X, name='SpectrogramCNN')

In [19]:
training()

Epoch 1/200
25/25 [==============================] - 6s 113ms/step - loss: 2.8698 - accuracy: 0.0915 - val_loss: 2.3581 - val_accuracy: 0.1000
Epoch 2/200
25/25 [==============================] - 2s 88ms/step - loss: 2.2884 - accuracy: 0.1316 - val_loss: 2.3239 - val_accuracy: 0.1000
Epoch 3/200
25/25 [==============================] - 2s 86ms/step - loss: 2.2539 - accuracy: 0.1479 - val_loss: 2.1724 - val_accuracy: 0.1200
Epoch 4/200
25/25 [==============================] - 2s 86ms/step - loss: 2.1504 - accuracy: 0.1967 - val_loss: 2.1903 - val_accuracy: 0.1800
Epoch 5/200
25/25 [==============================] - 2s 86ms/step - loss: 2.0047 - accuracy: 0.2381 - val_loss: 2.1197 - val_accuracy: 0.1800
Epoch 6/200
25/25 [==============================] - 2s 86ms/step - loss: 1.9779 - accuracy: 0.2444 - val_loss: 2.0624 - val_accuracy: 0.2000
Epoch 7/200
25/25 [==============================] - 2s 88ms/step - loss: 1.9028 - accuracy: 0.2895 - val_loss: 2.0376 - val_accuracy: 0.2100
Epoch

In [31]:
testing()

testing accuracy: 0.747474730014801
validation accuracy: 0.800000011920929
training accuracy: 0.829573929309845


In [25]:
#Save Weights in Model
def testing(model_name="mfcc_model.h5", epoch_name = 'best_weights.h5'):
    path = '/content/drive/MyDrive/mfcc_models/' + model_name
    path_best_epoch = '/content/drive/MyDrive/mfcc_models/' + epoch_name
    model = keras.models.load_model(path)
    model.load_weights(path_best_epoch)
    model.save('/content/drive/MyDrive/mfcc_models/mfcc_model.h5')
